In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Spark ML 3 quiz") \
        .config("spark.ui.port", "4050") \
        .getOrCreate()

sc = spark.sparkContext

In [24]:
df = spark.read.format('csv')\
               .option('delimiter', '\t')\
               .option('header', 'true')\
               .load('../../data/data-final.csv')
df.show(5)

+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+-------------------+-------+-------+-----------+----------+---------+---+-------+--------------------+---------------------+
|EXT1|EXT2|EXT3|EXT4|EXT5|EXT6|EXT7|EXT8|EXT9|EXT10|EST1|EST2|EST3|EST4|EST5|EST6|EST7|EST8|EST9|EST10|AGR1|AGR2|AGR3|AGR4|AGR5|AGR6|AGR7|AGR8|AGR9|AGR10|CSN1|CSN2|CSN3|CSN4|CSN5|CSN6|CSN7|CSN8|CSN9|CSN10|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|EXT1_E|E

In [25]:
df = df.filter((df.country == 'KR') | (df.country == 'US'))
df.show(5)

+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+-------------------+-------+-------+-----------+----------+---------+---+-------+--------------------+---------------------+
|EXT1|EXT2|EXT3|EXT4|EXT5|EXT6|EXT7|EXT8|EXT9|EXT10|EST1|EST2|EST3|EST4|EST5|EST6|EST7|EST8|EST9|EST10|AGR1|AGR2|AGR3|AGR4|AGR5|AGR6|AGR7|AGR8|AGR9|AGR10|CSN1|CSN2|CSN3|CSN4|CSN5|CSN6|CSN7|CSN8|CSN9|CSN10|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|EXT1_E|E

In [26]:
selected_columns = df.columns[:50] + ['country']
df = df.select(selected_columns)
df.show(5)

+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+
|EXT1|EXT2|EXT3|EXT4|EXT5|EXT6|EXT7|EXT8|EXT9|EXT10|EST1|EST2|EST3|EST4|EST5|EST6|EST7|EST8|EST9|EST10|AGR1|AGR2|AGR3|AGR4|AGR5|AGR6|AGR7|AGR8|AGR9|AGR10|CSN1|CSN2|CSN3|CSN4|CSN5|CSN6|CSN7|CSN8|CSN9|CSN10|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|country|
+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+
|   4|   3|   4|   3|   3|   3|   5|   3|   4|    3|   2|   4|   4|   2|   4|   2|   2|   2|   4|    4|   1|   2|   1|   5|   3|   5|   3|   4|   4|    5|   3|   2|   4|   2|   1|   4|   4|   2|   2|    5|

In [27]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

for i in range(50):
    df = df.withColumn(selected_columns[i], col(selected_columns[i]).cast(IntegerType()))

In [30]:
df = df.dropna()

In [31]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=selected_columns[:-1], outputCol='features')
df = assembler.transform(df)
df.select('features').show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[4.0,3.0,4.0,3.0,3.0,3.0,5.0,3.0,4.0,3.0,2.0,4.0,4.0,2.0,4.0,2.0,2.0,2.0,4.0,4.0,1.0,2.0,1.0,5.0,3.0,5.0,3.0,4.0,4.0,5.0,3.0,2.0,4.0,2.0,1.0,4.0,4.0,2.0,2.0,5.0,5.0,2.0,4.0,3.0,4.0,1.0,5.0,5.0,4.0,4.0]|
|[3.0,2.0,2.0,4.0,4.0,4.0,5.0,3.0,1.0,3.0,3.0,3.0,4.0,4.0,3.0,3.0,5.0,4.0,3.0,4.0,1.0,5.0,1.0,4.0,5.0,3.0,2.0,3.0,4.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,2.0,2.0,5.

In [32]:
import numpy as np
from pyspark.ml.feature import PCA
import matplotlib.pyplot as plt
from pyspark.ml.linalg import Vectors

pca = PCA(k=10, inputCol='features', outputCol='pca_features')
model = pca.fit(df)
reduced = model.transform(df)

22/11/03 17:28:23 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/11/03 17:28:23 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [35]:
reduced.select('pca_features').show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                                                                                                      |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[-0.04043380234523464,-10.050355253305987,-8.757222276330989,5.722797855895059,7.878904522749819,-16.693899067690083,0.14383312270588822,3.301126101327777,-0.6760480460571459,2.1869728774857657]|
|[3.221627954733662,-9.175316156554494,-7.490877378149124,0.2134712368605327,4.4525379347173315,-15.64895849496016,0.5251944643832874,4.197458422589124,0.01049880547707302,-0.7613988019967922]   |
|[4.69501703285

In [39]:
from pyspark.ml.clustering import GaussianMixture

gm = GaussianMixture(featuresCol='pca_features', k=2, tol=0.0001)
model = gm.fit(reduced)
prob = model.transform(reduced)
prob.show(5)

22/11/03 17:38:20 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/03 17:38:20 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+--------------------+--------------------+--------------------+----------+
|EXT1|EXT2|EXT3|EXT4|EXT5|EXT6|EXT7|EXT8|EXT9|EXT10|EST1|EST2|EST3|EST4|EST5|EST6|EST7|EST8|EST9|EST10|AGR1|AGR2|AGR3|AGR4|AGR5|AGR6|AGR7|AGR8|AGR9|AGR10|CSN1|CSN2|CSN3|CSN4|CSN5|CSN6|CSN7|CSN8|CSN9|CSN10|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|country|            features|        pca_features|         probability|prediction|
+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+--------------------+--------------------+----------------

In [43]:
prob_with_country = prob.select('probability', 'prediction', 'country')
prob_with_country.show(5, truncate=False)

+------------------------------------------+----------+-------+
|probability                               |prediction|country|
+------------------------------------------+----------+-------+
|[0.9991980531429148,8.019468570851764E-4] |0         |US     |
|[0.8383547702293657,0.16164522977063436]  |0         |US     |
|[0.9990350332417748,9.649667582251785E-4] |0         |US     |
|[0.9957631296687443,0.004236870331255682] |0         |US     |
|[0.9999432365733033,5.6763426696688696E-5]|0         |US     |
+------------------------------------------+----------+-------+
only showing top 5 rows



In [55]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.functions import vector_to_array
import pyspark.sql.functions as F

indexer = StringIndexer(inputCol='country', outputCol='country_index')
df_indexed = indexer.fit(prob_with_country).transform(prob_with_country)
df_indexed.show(5)

+--------------------+----------+-------+-------------+
|         probability|prediction|country|country_index|
+--------------------+----------+-------+-------------+
|[0.99919805314291...|         0|     US|          0.0|
|[0.83835477022936...|         0|     US|          0.0|
|[0.99903503324177...|         0|     US|          0.0|
|[0.99576312966874...|         0|     US|          0.0|
|[0.99994323657330...|         0|     US|          0.0|
+--------------------+----------+-------+-------------+
only showing top 5 rows



Exception in thread "serve RDD 577" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:458)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:556)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:524)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [83]:
ohe_encoder = OneHotEncoder(inputCol='country_index', outputCol='country_ohe', dropLast=False)
df_ohe = ohe_encoder.fit(df_indexed).transform(df_indexed)
df_ohe.show(5)

+--------------------+----------+-------+-------------+-------------+
|         probability|prediction|country|country_index|  country_ohe|
+--------------------+----------+-------+-------------+-------------+
|[0.99919805314291...|         0|     US|          0.0|(2,[0],[1.0])|
|[0.83835477022936...|         0|     US|          0.0|(2,[0],[1.0])|
|[0.99903503324177...|         0|     US|          0.0|(2,[0],[1.0])|
|[0.99576312966874...|         0|     US|          0.0|(2,[0],[1.0])|
|[0.99994323657330...|         0|     US|          0.0|(2,[0],[1.0])|
+--------------------+----------+-------+-------------+-------------+
only showing top 5 rows



In [87]:
assembler = VectorAssembler(inputCols=['probability', 'country_ohe'], outputCol='features')
result = assembler.transform(df_ohe)
result = result.select('features')
result.show(5, truncate=False)

+--------------------------------------------------+
|features                                          |
+--------------------------------------------------+
|[0.9991980531429148,8.019468570851764E-4,1.0,0.0] |
|[0.8383547702293657,0.16164522977063436,1.0,0.0]  |
|[0.9990350332417748,9.649667582251785E-4,1.0,0.0] |
|[0.9957631296687443,0.004236870331255682,1.0,0.0] |
|[0.9999432365733033,5.6763426696688696E-5,1.0,0.0]|
+--------------------------------------------------+
only showing top 5 rows



In [89]:
from pyspark.ml.stat import Correlation

corr = Correlation.corr(result, 'features', 'pearson').first()
corr

Row(pearson(features)=DenseMatrix(4, 4, [1.0, -1.0, 0.0086, -0.0086, -1.0, 1.0, -0.0086, 0.0086, 0.0086, -0.0086, 1.0, -1.0, -0.0086, 0.0086, -1.0, 1.0], False))

In [92]:
corr[0]

DenseMatrix(4, 4, [1.0, -1.0, 0.0086, -0.0086, -1.0, 1.0, -0.0086, 0.0086, 0.0086, -0.0086, 1.0, -1.0, -0.0086, 0.0086, -1.0, 1.0], False)